## Google News

In [2]:
import requests

def fetch_news(topic):
	url = f"https://google-news13.p.rapidapi.com/{topic}"

	querystring = {"lr":"en-IN", "hl":"en", "gl":"IN"}

	headers = {
		"x-rapidapi-key": "03a627dc4emshedbeb8b8893f5b9p190bacjsnaf5e8f2531c6",
		"x-rapidapi-host": "google-news13.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers, params=querystring)

	return response.json()

	# with open('news.json', 'w') as f:
	# 	f.write(response.text)

In [3]:
topic = input("Enter the topic: ")
response = fetch_news(topic)

for item in response['items']:
	title = item['title']
	link = item['newsUrl']
	print(title, link)
	break

# with open('news.json', 'w') as f:
# 	f.write(response.text)

Gautam Adani takes major step as Adani Power cuts power supply to Bangladesh due to.... https://www.india.com/business/gautam-adani-takes-major-step-as-adani-power-cuts-power-supply-to-bangladesh-due-to-outstanding-bills-846-million-dollars-adani-group-7365019/


In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()


NEWSCATCHER_API_KEY = os.getenv('NEWSCATCHER_API_KEY')
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
SERPHOUSE_API_KEY = os.getenv('SERPHOUSE_API_KEY')

In [5]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

llm = ChatGroq(
    groq_api_key = GROQ_API_KEY,
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
    streaming=True,
)

def summarize(title, link):
	prompt = PromptTemplate(
		input_variables=["title", "link"],
		template="""
		You are an AI assistant. Your task is to read the news title and visit the provided news link. 
		After visiting the link, summarize the content of the news article.

		News Title: {title}
		News Link: {link}

		Please provide a concise summary of the article in markdown format and exclude the output preamble.
		"""
	)
	chain = prompt | llm
	res = chain.invoke({"title": title, "link": link})
	return res.content

In [6]:
x = summarize("Harmanpreet Kaur-led India shown no mercy after 'pathetic loss' vs NZ, face brutal online backlash over old BCCI tweet", "https://www.hindustantimes.com/cricket/harmanpreet-kaur-india-no-mercy-pathetic-loss-womens-t20-world-cup-nz-loss-face-brutal-online-backlash-old-bcci-tweet-101728122092807.html")
print(x)

**India Women's Cricket Team Faces Backlash After Loss to New Zealand**

The Indian women's cricket team, led by Harmanpreet Kaur, faced severe criticism and online backlash after their loss to New Zealand in the Women's T20 World Cup. The team's performance was described as "pathetic" by many, including former players and cricket experts.

The backlash was further fueled by an old tweet from the Board of Control for Cricket in India (BCCI), which had stated that the team's performance was a "source of pride" for the country. The tweet was seen as insensitive and hypocritical, given the team's recent performance.

The online criticism was brutal, with many calling for the team's captain, Harmanpreet Kaur, to be held accountable for the team's poor performance. The team's coach, Ramesh Powar, also faced criticism for his tactics and decision-making during the match.

The incident highlights the intense scrutiny and pressure that the Indian women's cricket team faces, both on and off the

## SerpHouse API

In [2]:
import requests
import json
url = "https://api.serphouse.com/serp/live"

payload = {
  "data":{
			"q": "Sports news on 1st October 2023",
			"domain": "google.com",
			"loc": "Kolkata,West Bengal,India",
			"lang": "en",
			"device": "desktop",
			"serp_type": "news",
			"page": "1",
			"verbatim": "1"
	}
}
headers = {
    'accept': "application/json",
    'content-type': "application/json",
    'authorization': "Bearer {}".format(SERPHOUSE_API_KEY)
}

response = requests.request("POST", url, json=payload, headers=headers)
news = response.json()

In [3]:
# news = news['results']['results']['news']
news

{'status': 'error',
 'msg': 'Your SERPHouse account has either run out of available credits (try upgrading your Plan), or there is a payment problem.',
 'error': ''}

In [11]:
# for i in news:
# 	print(f"{i['title']}, {i['url']}")

# print("Total news fetched: ", len(news))

## NewsCatcher API

In [4]:
import os
# NEWSCATCHER_API_KEY = os.getenv('NEWSCATCHER_API_KEY')
# GROQ_API_KEY = os.getenv('GROQ_API_KEY')

import requests

url = "https://api.newscatcherapi.com/v2/search"

querystring = {"q":"*", "topic":"business", "lang":"en","sort_by":"relevancy","page":"1","from":"Yesterday","to":"Today", "countries":"IN"}

headers = {"x-api-key": NEWSCATCHER_API_KEY}

response = requests.request("GET", url, headers=headers, params=querystring)

news = response.json()

print(news)

{'status': 'ok', 'total_hits': 699, 'page': 1, 'total_pages': 14, 'page_size': 50, 'articles': [{'title': 'Madhur Bhandarkar calls tabla maestro Zakir Hussain\'s demise "a void that no one can fill"', 'author': 'ANI', 'published_date': '2024-12-16 17:17:00', 'published_date_precision': 'timezone unknown', 'link': 'https://timesofindia.indiatimes.com/entertainment/hindi/bollywood/news/madhur-bhandarkar-calls-tabla-maestro-zakir-hussains-demise-a-void-that-no-one-can-fill/articleshow/116372293.cms', 'clean_url': 'indiatimes.com', 'excerpt': "In a heartfelt tribute to the legendary tabla virtuoso Ustad Zakir Hussain, filmmaker Madhur Bhandarkar expressed deep sorrow over the maestro's passi", 'summary': 'In a heartfelt tribute to the legendary tabla virtuoso Ustad\nZakir Hussain\n, filmmaker\nMadhur Bhandarkar\nexpressed deep sorrow over the maestro\'s passing.\nBhandarkar, who had long admired Hussain\'s musical brilliance, spoke to and mourned the loss, calling it a deeply emotional mom

In [5]:
news['articles'][0]['title']

'Madhur Bhandarkar calls tabla maestro Zakir Hussain\'s demise "a void that no one can fill"'

In [12]:
'articles' in news

True

In [19]:
from newscatcherapi import NewsCatcherApiClient

newscatcherapi = NewsCatcherApiClient(x_api_key=NEWSCATCHER_API_KEY)

news_articles = newscatcherapi.get_search(q="Tesla", topic="tech", lang="hi", sort_by="relevancy", page=1, countries="IN")

In [20]:
news_articles

{'status': 'No matches for your search.',
 'total_hits': 0,
 'page': 0,
 'total_pages': 0,
 'page_size': 0,
 'user_input': {'q': 'Tesla',
  'search_in': ['title_summary'],
  'lang': ['hi'],
  'not_lang': None,
  'countries': ['IN'],
  'not_countries': None,
  'from': '2024-11-28 00:00:00',
  'to': None,
  'ranked_only': 'True',
  'from_rank': None,
  'to_rank': None,
  'is_opinion': None,
  'is_republisher': None,
  'sort_by': 'relevancy',
  'page': 1,
  'size': 50,
  'sources': None,
  'not_sources': [],
  'topic': 'tech',
  'published_date_precision': None}}

In [13]:
for i in news['articles']:
	print(f"{i['title']}, {i['link']}")

BAU developing business modules for farmers, https://timesofindia.indiatimes.com/city/patna/baus-innovative-business-module-set-to-transform-farming-in-india/articleshow/114864177.cms
AO Smith completes ₹600-crore acquisition of HUL's water purification business Pureit, https://www.livemint.com/companies/ao-smith-completes-rs-600-crore-acquisition-of-huls-water-purification-business-pureit-11730473644073.html
Sona expects Escorts' railway parts business to be accretive from the first year, https://auto.economictimes.indiatimes.com/news/auto-components/sona-expects-escorts-railway-parts-business-to-be-accretive-from-the-first-year/114847569
Greaves Cotton appoints Ramachandra Puttanna as Business Head for EV solutions and new ventures, https://auto.economictimes.indiatimes.com/news/industry/greaves-cotton-appoints-ramachandra-puttanna-as-business-head-for-ev-solutions-and-new-ventures/114840069
Thane: Affordability And Strategic Location Propel City Into Booming Commercial Hub, https://